In [2]:
import pandas as pd


In [33]:
# Load datasets

streams = pd.read_csv("streams.csv")
tracks = pd.read_csv("tracks.csv")
users = pd.read_csv("users.csv")

In [34]:
# Prepare data

streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.date
streams['listen_hour'] = pd.to_datetime(streams['listen_time']).dt.hour
full_data = streams.merge(tracks,on='track_id').merge(users,on='user_id')
full_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744143 entries, 0 to 744142
Data columns (total 29 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           744143 non-null  int64  
 1   track_id          744143 non-null  object 
 2   listen_time       744143 non-null  object 
 3   listen_date       744143 non-null  object 
 4   listen_hour       744143 non-null  int32  
 5   id                744143 non-null  int64  
 6   artists           744143 non-null  object 
 7   album_name        744143 non-null  object 
 8   track_name        744143 non-null  object 
 9   popularity        744143 non-null  int64  
 10  duration_ms       744143 non-null  int64  
 11  explicit          744143 non-null  bool   
 12  danceability      744143 non-null  float64
 13  energy            744143 non-null  float64
 14  key               744143 non-null  int64  
 15  loudness          744143 non-null  float64
 16  mode              74

In [35]:
# Hourly Unique Listeners

hourly_unique_listeners = full_data.groupby(['listen_date','listen_hour'])['user_id'].nunique().reset_index(name='unique_listeners')



In [36]:
# Top Listened Artist of the Hour

artist_listen_counts = full_data.groupby(['listen_date','listen_hour','artists']).size().reset_index(name='listen_counts')
top_artist = artist_listen_counts.loc[artist_listen_counts.groupby(['listen_date', 'listen_hour'])['listen_counts'].idxmax()]
top_artist = top_artist.rename(columns={'artist':'top_artist'})
top_artist

,listen_date,listen_hour,artists,listen_counts
7,2024-01-01,0,Denise Hernandez,7
64,2024-01-01,1,Joseph White,6
92,2024-01-01,2,Brenda Ramirez,5
130,2024-01-01,3,Bradley Taylor,6
211,2024-01-01,4,Victoria Bradshaw,9
...,...,...,...,...
312807,2024-11-19,11,Kelly Frazier,5
312866,2024-11-19,12,Wendy Durham,5
312887,2024-11-19,13,Mr. Nicholas Stone,5
312903,2024-11-19,14,Angel Arellano,7


In [43]:
full_data['session_id'] = full_data['user_id'].astype(str) + '-' + full_data['listen_time'].astype(str)
sessions_per_user = full_data.groupby(['listen_date', 'listen_hour', 'user_id']).nunique('session_id').reset_index()
avg_sessions_per_user = sessions_per_user.groupby(['listen_date', 'listen_hour'])['session_id'].mean().reset_index(name='avg_sessions_per_user')


In [44]:
track_diversity = full_data.groupby(['listen_date', 'listen_hour'])['track_id'].agg(['nunique', 'count']).reset_index()
track_diversity['diversity_index'] = track_diversity['nunique'] / track_diversity['count']


In [42]:
users['age_group']= pd.cut(users['user_age'],bins=[0,25,35,45,55,65,100],labels=['0-25','26-35','36-45','46-55','56-65','65+'],right=False)
user_group_engagement = full_data.merge(users,on='user_id').groupby(['listen_date','listen_hour','age_group']).size().reset_index(name='streams')
most_engaged_group = user_group_engagement.loc[user_group_engagement.groupby(['listen_date', 'listen_hour'])['streams'].idxmax()].rename(columns={'age_group': 'most_engaged_age_group'})
most_engaged_group


/tmp/ipykernel_8242/3061787790.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  user_group_engagement = full_data.merge(users,on='user_id').groupby(['listen_date','listen_hour','age_group']).size().reset_index(name='streams')


,listen_date,listen_hour,most_engaged_age_group,streams
2,2024-01-01,0,36-45,77
9,2024-01-01,1,46-55,79
15,2024-01-01,2,46-55,52
22,2024-01-01,3,56-65,73
26,2024-01-01,4,36-45,57
...,...,...,...,...
46626,2024-11-19,19,0-25,0
46632,2024-11-19,20,0-25,0
46638,2024-11-19,21,0-25,0
46644,2024-11-19,22,0-25,0


In [45]:
final_kpis = hourly_unique_listeners.merge(top_artist, on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(avg_sessions_per_user, on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(track_diversity[['listen_date', 'listen_hour', 'diversity_index']], on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(most_engaged_group[['listen_date', 'listen_hour', 'most_engaged_age_group']], on=['listen_date', 'listen_hour'])


In [46]:
final_kpis.head()


,listen_date,listen_hour,unique_listeners,artists,listen_counts,avg_sessions_per_user,diversity_index,most_engaged_age_group
0,2024-01-01,0,41,Denise Hernandez,7,1.0,0.384615,36-45
1,2024-01-01,1,46,Joseph White,6,1.0,0.425926,46-55
2,2024-01-01,2,42,Brenda Ramirez,5,1.0,0.466667,46-55
3,2024-01-01,3,50,Bradley Taylor,6,1.0,0.418803,56-65
4,2024-01-01,4,40,Victoria Bradshaw,9,1.0,0.382979,36-45
